# Actividad de seguimiento - Simulación 2

|Integrante|correo|usuario github|
|---|---|---|
|David Camilo García Echavarría|davidc.garcia@udea.edu.co|davidc-garciae|
|Cristian David Tamayo Espinosa|cristian.tamayoe@udea.edu.co|CrissPr|

## Instrucciones

Antes de empezar a realizar esta actividad haga un **fork** de este repositorio y sobre este trabaje en la solución de las preguntas planteadas en la actividad de simulación. Las respuestas deben ser respondidas en español o si lo prefiere en ingles en el lugar señalado para ello (La palabra **answer** muestra donde).


## Homework (Simulation)

This program, [mlfq.py](mlfq.py), allows you to see how the MLFQ scheduler presented in this chapter behaves. See the [README](https://github.com/remzi-arpacidusseau/ostep-homework/blob/master/cpu-sched-mlfq/README.md) for details.

In [ ]:
%pip install bash_kernel
!python -m bash_kernel.install

## Questions

#### 1. Run a few randomly-generated problems with just two jobs and two queues; compute the MLFQ execution trace for each. Make your life easier by limiting the length of each job and turning off I/Os.

In [24]:
!python mlfq.py -j 2 -n 2 -M 0 -s 0 -m 30 -c

Here is the list of inputs:
OPTIONS jobs 2
OPTIONS queues 2
OPTIONS allotments for queue  1 is   1
OPTIONS quantum length for queue  1 is  10
OPTIONS allotments for queue  0 is   1
OPTIONS quantum length for queue  0 is  10
OPTIONS boost 0
OPTIONS ioTime 5
OPTIONS stayAfterIO False
OPTIONS iobump False


For each job, three defining characteristics are given:
  startTime : at what time does the job enter the system
  runTime   : the total CPU time needed by the job to finish
  ioFreq    : every ioFreq time units, the job issues an I/O
              (the I/O takes ioTime units to complete)

Job List:
  Job  0: startTime   0 - runTime  25 - ioFreq   0
  Job  1: startTime   0 - runTime  13 - ioFreq   0


Execution Trace:

[ time 0 ] JOB BEGINS by JOB 0
[ time 0 ] JOB BEGINS by JOB 1
[ time 0 ] Run JOB 0 at PRIORITY 1 [ TICKS 9 ALLOT 1 TIME 24 (of 25) ]
[ time 1 ] Run JOB 0 at PRIORITY 1 [ TICKS 8 ALLOT 1 TIME 23 (of 25) ]
[ time 2 ] Run JOB 0 at PRIORITY 1 [ TICKS 7 ALLOT 1 TIME 22 (of 25

Answer

**Explicación:**

-   `-j 2`: Dos trabajos.
    
-   `-n 2`: Dos colas.
    
-   `-m 30`: Limita el tiempo máximo de ejecución de cada trabajo a 30 ms.
    
-   `-M 0`: Desactiva I/O.
    
-   `-s <semilla>`: Usa una semilla aleatoria (ej. `-s 1`, `-s 2`, etc.).
    
-   `-c`: Muestra la solución.

**Estadísticas Finales:**

-   **Job 0:**
    -   `response`: 0 (Empezó a ejecutarse inmediatamente al llegar en T=0).
    -   `turnaround`: 38 (Tiempo desde que llegó T=0 hasta que finalizó T=38).
-   **Job 1:**
    -   `response`: 10 (Llegó en T=0, pero empezó a ejecutarse en T=10).
    -   `turnaround`: 33 (Tiempo desde que llegó T=0 hasta que finalizó T=33).
-   **Promedios:**
    -   `response`: (0 + 10) / 2 = 5.00 ms
    -   `turnaround`: (38 + 33) / 2 = 35.50 ms

**Análisis:**

-   El planificador MLFQ priorizó inicialmente ambos trabajos por igual (Q1).

-   Job 0, al ejecutarse primero, consumió su tiempo en la cola alta, lo que le dio un excelente tiempo de respuesta (0) pero retrasó el inicio de Job 1.

-   Ambos trabajos fueron degradados a la cola de baja prioridad (Q0) después de usar su `allotment` inicial.

-   En la cola baja, se aplicó Round-Robin.

-   El trabajo más corto (Job 1) terminó antes (T=33 vs T=38), lo cual es un objetivo del MLFQ (aproximar SJF), aunque su tiempo de respuesta fue peor en este caso específico.

-   La ausencia de I/O y `boost` hace que el comportamiento sea puramente basado en el consumo de CPU y las reglas de descenso de prioridad.



#### 2. How would you run the scheduler to reproduce each of the examples in the chapter?

Answer

<img src="8.2.PNG">

- Ejemplo 1: **Figura 8.2 (Trabajo largo único):** Un trabajo que consume tiempo y baja de prioridad. 3 colas, quantum 10ms.

In [25]:
# Un trabajo (implícito con jlist), empieza en 0, dura 100ms, sin E/S (0)
# 3 colas (-n 3), quantum 10ms (-q 10), mostrar solución (-c)
!python mlfq.py --jlist 0,100,0 -n 3 -q 10 -c

Here is the list of inputs:
OPTIONS jobs 1
OPTIONS queues 3
OPTIONS allotments for queue  2 is   1
OPTIONS quantum length for queue  2 is  10
OPTIONS allotments for queue  1 is   1
OPTIONS quantum length for queue  1 is  10
OPTIONS allotments for queue  0 is   1
OPTIONS quantum length for queue  0 is  10
OPTIONS boost 0
OPTIONS ioTime 5
OPTIONS stayAfterIO False
OPTIONS iobump False


For each job, three defining characteristics are given:
  startTime : at what time does the job enter the system
  runTime   : the total CPU time needed by the job to finish
  ioFreq    : every ioFreq time units, the job issues an I/O
              (the I/O takes ioTime units to complete)

Job List:
  Job  0: startTime   0 - runTime 100 - ioFreq   0


Execution Trace:

[ time 0 ] JOB BEGINS by JOB 0
[ time 0 ] Run JOB 0 at PRIORITY 2 [ TICKS 9 ALLOT 1 TIME 99 (of 100) ]
[ time 1 ] Run JOB 0 at PRIORITY 2 [ TICKS 8 ALLOT 1 TIME 98 (of 100) ]
[ time 2 ] Run JOB 0 at PRIORITY 2 [ TICKS 7 ALLOT 1 TIME 97 (of 

- _Análisis:_ El trabajo empieza en Q2, tras 10ms baja a Q1, tras otros 10ms baja a Q0 y permanece allí.

<img src="8.3.PNG">

- Ejemplo 2: **Figura 8.3 Izquierda (Llega un trabajo corto):** Job A largo, Job B corto llega en T=100. 3 colas, quantum 10ms.



In [26]:
# Job 0: empieza 0, dura 180ms, sin E/S. Job 1: empieza 100, dura 20ms, sin E/S.
# Quantum 10ms (-q 10), 3 colas (por defecto), mostrar solución (-c)
!python mlfq.py --jlist 0,180,0:100,20,0 -q 10 -c

Here is the list of inputs:
OPTIONS jobs 2
OPTIONS queues 3
OPTIONS allotments for queue  2 is   1
OPTIONS quantum length for queue  2 is  10
OPTIONS allotments for queue  1 is   1
OPTIONS quantum length for queue  1 is  10
OPTIONS allotments for queue  0 is   1
OPTIONS quantum length for queue  0 is  10
OPTIONS boost 0
OPTIONS ioTime 5
OPTIONS stayAfterIO False
OPTIONS iobump False


For each job, three defining characteristics are given:
  startTime : at what time does the job enter the system
  runTime   : the total CPU time needed by the job to finish
  ioFreq    : every ioFreq time units, the job issues an I/O
              (the I/O takes ioTime units to complete)

Job List:
  Job  0: startTime   0 - runTime 180 - ioFreq   0
  Job  1: startTime 100 - runTime  20 - ioFreq   0


Execution Trace:

[ time 0 ] JOB BEGINS by JOB 0
[ time 0 ] Run JOB 0 at PRIORITY 2 [ TICKS 9 ALLOT 1 TIME 179 (of 180) ]
[ time 1 ] Run JOB 0 at PRIORITY 2 [ TICKS 8 ALLOT 1 TIME 178 (of 180) ]
[ time 2 ] R

-   _Análisis:_ Job A corre, baja a Q0. En T=100 llega Job B a Q2. Job B se ejecuta (intercalándose si hay otros trabajos en Q2, pero aquí no) por 2 quantums (20ms) y termina antes de bajar de prioridad. Job A continúa en Q0. MLFQ aproxima SJF dando prioridad alta a B.

- Ejemplo 3: **Figura 8.3 Derecha (Trabajo con E/S):** Job A largo, Job B interactivo (1ms CPU, luego E/S). Se asume la regla 4b original (permanece en la misma prioridad si hace E/S antes de agotar quantum/allotment). Tiempo de E/S 5ms.





In [27]:
# Job 0: largo. Job 1: interactivo (1ms CPU, E/S), llega en 100.
# Quantum 10ms, 3 colas, E/S dura 5ms (-i 5).
# Usar regla antigua (-S) para que B se quede arriba. Mostrar solución (-c)
!python mlfq.py --jlist 0,180,0:100,20,1 -q 10 -i 5 -S -c

Here is the list of inputs:
OPTIONS jobs 2
OPTIONS queues 3
OPTIONS allotments for queue  2 is   1
OPTIONS quantum length for queue  2 is  10
OPTIONS allotments for queue  1 is   1
OPTIONS quantum length for queue  1 is  10
OPTIONS allotments for queue  0 is   1
OPTIONS quantum length for queue  0 is  10
OPTIONS boost 0
OPTIONS ioTime 5
OPTIONS stayAfterIO True
OPTIONS iobump False


For each job, three defining characteristics are given:
  startTime : at what time does the job enter the system
  runTime   : the total CPU time needed by the job to finish
  ioFreq    : every ioFreq time units, the job issues an I/O
              (the I/O takes ioTime units to complete)

Job List:
  Job  0: startTime   0 - runTime 180 - ioFreq   0
  Job  1: startTime 100 - runTime  20 - ioFreq   1


Execution Trace:

[ time 0 ] JOB BEGINS by JOB 0
[ time 0 ] Run JOB 0 at PRIORITY 2 [ TICKS 9 ALLOT 1 TIME 179 (of 180) ]
[ time 1 ] Run JOB 0 at PRIORITY 2 [ TICKS 8 ALLOT 1 TIME 178 (of 180) ]
[ time 2 ] Ru

-   _Análisis:_ Job A corre y baja a Q0. Llega Job B (T=100) a Q2. Corre 1ms, hace E/S (dura 5ms). Como usó `-S`, al volver, sigue en Q2. Job A corre mientras B está en E/S. B vuelve y preempta a A. B se mantiene en alta prioridad porque cede la CPU antes de agotar su quantum. _Nota: Sin `-S`, B bajaría de prioridad después de acumular 10ms de CPU en Q2, independientemente de las E/S._

<img src="8.4.PNG">
    
- Ejemplo 4: **Figura 8.4 (Priority Boost):** Izquierda (sin boost, inanición): Job largo (A), 2 interactivos cortos (B, C) llegan y acaparan CPU. Derecha (con boost): Lo mismo, pero con boost cada 100ms.


In [28]:
# Izquierda (Sin Boost): Job A (0,200,0), Job B (50,30,1), Job C (50,30,1)
# Quantum 10ms, E/S 5ms. Mostrar solución (-c)
#!python mlfq.py --jlist 0,200,0:50,30,1:50,30,1 -q 10 -i 5 -c

# Derecha (Con Boost cada 100ms): Añadir -B 100
!python mlfq.py --jlist 0,200,0:50,30,1:50,30,1 -q 10 -i 5 -B 100 -c

Here is the list of inputs:
OPTIONS jobs 3
OPTIONS queues 3
OPTIONS allotments for queue  2 is   1
OPTIONS quantum length for queue  2 is  10
OPTIONS allotments for queue  1 is   1
OPTIONS quantum length for queue  1 is  10
OPTIONS allotments for queue  0 is   1
OPTIONS quantum length for queue  0 is  10
OPTIONS boost 100
OPTIONS ioTime 5
OPTIONS stayAfterIO False
OPTIONS iobump False


For each job, three defining characteristics are given:
  startTime : at what time does the job enter the system
  runTime   : the total CPU time needed by the job to finish
  ioFreq    : every ioFreq time units, the job issues an I/O
              (the I/O takes ioTime units to complete)

Job List:
  Job  0: startTime   0 - runTime 200 - ioFreq   0
  Job  1: startTime  50 - runTime  30 - ioFreq   1
  Job  2: startTime  50 - runTime  30 - ioFreq   1


Execution Trace:

[ time 0 ] JOB BEGINS by JOB 0
[ time 0 ] Run JOB 0 at PRIORITY 2 [ TICKS 9 ALLOT 1 TIME 199 (of 200) ]
[ time 1 ] Run JOB 0 at PRIORITY

-   _Análisis:_ Sin boost, una vez que B y C llegan (T=50), se ejecutan en Q2 (o Q1 si bajan un nivel) preemptando a A (en Q0), y A sufre inanición. Con boost (`-B 100`), cada 100ms todos los trabajos (incluido A) vuelven a Q2, garantizando que A obtenga algo de CPU periódicamente.

<img src="8.5.PNG">
    
- Ejemplo 5: **Figura 8.5 (Gaming):** Izquierda (con gaming): Job B hace E/S justo antes de terminar su quantum para quedarse arriba (usa `-S`). Derecha (sin gaming): Mismo workload, pero con reglas nuevas (sin `-S`). 2 colas, quantum 10, allotment 1.





In [29]:
# Izquierda (Gaming): Job A (largo), Job B (llega T=1, E/S cada 9ms).
# 2 colas (-n 2), quantum 10 (-q 10), allotment 1 (-a 1), E/S 1ms (-i 1), regla antigua (-S), solución (-c)
!python mlfq.py --jlist 0,200,0:1,100,9 -n 2 -q 10 -a 1 -i 1 -S -c

# Derecha (Sin Gaming): Quitar -S
#!python mlfq.py --jlist 0,200,0:1,100,9 -n 2 -q 10 -a 1 -i 1 -c

Here is the list of inputs:
OPTIONS jobs 2
OPTIONS queues 2
OPTIONS allotments for queue  1 is   1
OPTIONS quantum length for queue  1 is  10
OPTIONS allotments for queue  0 is   1
OPTIONS quantum length for queue  0 is  10
OPTIONS boost 0
OPTIONS ioTime 1
OPTIONS stayAfterIO True
OPTIONS iobump False


For each job, three defining characteristics are given:
  startTime : at what time does the job enter the system
  runTime   : the total CPU time needed by the job to finish
  ioFreq    : every ioFreq time units, the job issues an I/O
              (the I/O takes ioTime units to complete)

Job List:
  Job  0: startTime   0 - runTime 200 - ioFreq   0
  Job  1: startTime   1 - runTime 100 - ioFreq   9


Execution Trace:

[ time 0 ] JOB BEGINS by JOB 0
[ time 0 ] Run JOB 0 at PRIORITY 1 [ TICKS 9 ALLOT 1 TIME 199 (of 200) ]
[ time 1 ] JOB BEGINS by JOB 1
[ time 1 ] Run JOB 0 at PRIORITY 1 [ TICKS 8 ALLOT 1 TIME 198 (of 200) ]
[ time 2 ] Run JOB 0 at PRIORITY 1 [ TICKS 7 ALLOT 1 TIME 197 (o

-   _Análisis:_ Con `-S`, Job B corre 9ms en Q1, hace E/S (1ms), vuelve a Q1 con allotment reseteado. Repite esto, monopolizando casi toda la CPU mientras está activo, dejando a A correr solo durante la E/S de B. Sin `-S` (regla 4 moderna), la cuenta del tiempo de CPU de B en Q1 se acumula. Después de usar 10ms en total en Q1 (p.ej., 9ms + 1ms tras volver de E/S), B baja a Q0, permitiendo una distribución más justa.

<img src="8.6.PNG">
    
- Ejemplo 6: **Figura 8.6 (Quantums variables):** Prioridades más bajas tienen quantums más largos. 3 colas, Q2 quantum 10, Q1 quantum 20, Q0 quantum 40. Allotments: 2 slices en Q2 (20ms), 2 slices en Q1 (40ms).





In [30]:
# 2 trabajos largos (-j 2 -m 200 -M 0)
# Quantums por cola (-Q 10,20,40), Allotments por cola (-A 2,2,1)
# El allotment se da en #slices. Q2: 2*10=20ms, Q1: 2*20=40ms, Q0: 1*40=40ms
!python mlfq.py -j 2 -m 200 -M 0 -Q 10,20,40 -A 2,2,1 -c

Here is the list of inputs:
OPTIONS jobs 2
OPTIONS queues 3
OPTIONS allotments for queue  2 is   2
OPTIONS quantum length for queue  2 is  10
OPTIONS allotments for queue  1 is   2
OPTIONS quantum length for queue  1 is  20
OPTIONS allotments for queue  0 is   1
OPTIONS quantum length for queue  0 is  40
OPTIONS boost 0
OPTIONS ioTime 5
OPTIONS stayAfterIO False
OPTIONS iobump False


For each job, three defining characteristics are given:
  startTime : at what time does the job enter the system
  runTime   : the total CPU time needed by the job to finish
  ioFreq    : every ioFreq time units, the job issues an I/O
              (the I/O takes ioTime units to complete)

Job List:
  Job  0: startTime   0 - runTime 169 - ioFreq   0
  Job  1: startTime   0 - runTime  84 - ioFreq   0


Execution Trace:

[ time 0 ] JOB BEGINS by JOB 0
[ time 0 ] JOB BEGINS by JOB 1
[ time 0 ] Run JOB 0 at PRIORITY 2 [ TICKS 9 ALLOT 2 TIME 168 (of 169) ]
[ time 1 ] Run JOB 0 at PRIORITY 2 [ TICKS 8 ALLOT 2 T

- _Análisis:_ Los trabajos permanecen más tiempo en las colas de menor prioridad antes de que ocurra un cambio de contexto (Round Robin dentro de la cola). El primer trabajo corre 10ms en Q2, luego el segundo 10ms. Ambos usan su primer slice. Luego el primero otros 10ms (total 20ms, allotment de Q2 agotado), baja a Q1. El segundo otros 10ms, baja a Q1. Ahora en Q1 (quantum 20), el primero corre 20ms, luego el segundo 20ms. Usan el primer slice de Q1. El primero otros 20ms (total 40ms, allotment de Q1 agotado), baja a Q0. El segundo otros 20ms, baja a Q0. En Q0 (quantum 40), se alternan en bloques de 40ms.

#### 3. How would you configure the scheduler parameters to behave just like a round-robin scheduler?



In [31]:
!python mlfq.py -n 1 -q 10 -a 1000 -B 0 --jlist "0,50,0:10,50,0" -c

Here is the list of inputs:
OPTIONS jobs 2
OPTIONS queues 1
OPTIONS allotments for queue  0 is 1000
OPTIONS quantum length for queue  0 is  10
OPTIONS boost 0
OPTIONS ioTime 5
OPTIONS stayAfterIO False
OPTIONS iobump False


For each job, three defining characteristics are given:
  startTime : at what time does the job enter the system
  runTime   : the total CPU time needed by the job to finish
  ioFreq    : every ioFreq time units, the job issues an I/O
              (the I/O takes ioTime units to complete)

Job List:
  Job  0: startTime   0 - runTime  50 - ioFreq   0
  Job  1: startTime  10 - runTime  50 - ioFreq   0


Execution Trace:

[ time 0 ] JOB BEGINS by JOB 0
[ time 0 ] Run JOB 0 at PRIORITY 0 [ TICKS 9 ALLOT 1000 TIME 49 (of 50) ]
[ time 1 ] Run JOB 0 at PRIORITY 0 [ TICKS 8 ALLOT 1000 TIME 48 (of 50) ]
[ time 2 ] Run JOB 0 at PRIORITY 0 [ TICKS 7 ALLOT 1000 TIME 47 (of 50) ]
[ time 3 ] Run JOB 0 at PRIORITY 0 [ TICKS 6 ALLOT 1000 TIME 46 (of 50) ]
[ time 4 ] Run JOB 0 at P

In [ ]:
execution_trace = [
    "[ time 0 ] Run JOB 0 at PRIORITY 1 [ TICKS 9 ALLOT 1 TIME 24 (of 25) ]",
    "[ time 1 ] Run JOB 0 at PRIORITY 1 [ TICKS 8 ALLOT 1 TIME 23 (of 25) ]",
    # ... más líneas del trace ...
    "[ time 38 ] FINISHED JOB 0"
]

for line in execution_trace:
    try:
        if "Run JOB" in line:
            parts = line.split()
            print(parts)
            time = int(parts[2])  # Cambiado a parts[2] para extraer el tiempo correctamente
            job_id = int(parts[6])
            priority = int(parts[9])
            print(f"Time: {time}, Job ID: {job_id}, Priority: {priority}")
    except ValueError as e:
        print(f"Error al procesar la línea: {line}")
        print(f"Detalles del error: {e}")

['[', 'time', '0', ']', 'Run', 'JOB', '0', 'at', 'PRIORITY', '1', '[', 'TICKS', '9', 'ALLOT', '1', 'TIME', '24', '(of', '25)', ']']
Error al procesar la línea: [ time 0 ] Run JOB 0 at PRIORITY 1 [ TICKS 9 ALLOT 1 TIME 24 (of 25) ]
Detalles del error: invalid literal for int() with base 10: 'time'
['[', 'time', '1', ']', 'Run', 'JOB', '0', 'at', 'PRIORITY', '1', '[', 'TICKS', '8', 'ALLOT', '1', 'TIME', '23', '(of', '25)', ']']
Error al procesar la línea: [ time 1 ] Run JOB 0 at PRIORITY 1 [ TICKS 8 ALLOT 1 TIME 23 (of 25) ]
Detalles del error: invalid literal for int() with base 10: 'time'


Answer

-   **Explicación:**
    
    -   `-n 1`: Una sola cola.
        
    -   `-q 10`: Quantum de 10 ms.
        
    -   `-a 1000`: Allotment muy alto para evitar cambios de prioridad.
        
    -   `-B 0`: Sin priority boost.
        
    -   `--jlist "0,50,0:10,50,0"`: Dos trabajos de 50 ms sin E/S.
        

**Análisis:**

-   Todos los trabajos permanecen en la misma cola y se alternan cada 10 ms (RR).
    
-   Sin cambios de prioridad ni boosts, el comportamiento es idéntico a Round Robin.

#### 4. Craft a workload with two jobs and scheduler parameters so that one job takes advantage of the older Rules 4a and 4b (turned on with the -S flag) to game the scheduler and obtain 99% of the CPU over a particular time interval.



Answer

-   **Explicación:**
    
    -   `-n 1`: Una sola cola.
        
    -   `-q 10`: Quantum de 10 ms.
        
    -   `-a 1000`: Allotment muy alto para evitar cambios de prioridad.
        
    -   `-B 0`: Sin priority boost.
        
    -   `--jlist "0,50,0:10,50,0"`: Dos trabajos de 50 ms sin E/S.
        

**Análisis:**

-   Todos los trabajos permanecen en la misma cola y se alternan cada 10 ms (RR).
    
-   Sin cambios de prioridad ni boosts, el comportamiento es idéntico a Round Robin.

#### 4. Craft a workload with two jobs and scheduler parameters so that one job takes advantage of the older Rules 4a and 4b (turned on with the -S flag) to game the scheduler and obtain 99% of the CPU over a particular time interval.



Answer

-   **Explicación:**
    
    -   `-n 1`: Una sola cola.
        
    -   `-q 10`: Quantum de 10 ms.
        
    -   `-a 1000`: Allotment muy alto para evitar cambios de prioridad.
        
    -   `-B 0`: Sin priority boost.
        
    -   `--jlist "0,50,0:10,50,0"`: Dos trabajos de 50 ms sin E/S.
        

**Análisis:**

-   Todos los trabajos permanecen en la misma cola y se alternan cada 10 ms (RR).
    
-   Sin cambios de prioridad ni boosts, el comportamiento es idéntico a Round Robin.

#### 4. Craft a workload with two jobs and scheduler parameters so that one job takes advantage of the older Rules 4a and 4b (turned on with the -S flag) to game the scheduler and obtain 99% of the CPU over a particular time interval.



Answer

-   **Explicación:**
    
    -   `-n 1`: Una sola cola.
        
    -   `-q 10`: Quantum de 10 ms.
        
    -   `-a 1000`: Allotment muy alto para evitar cambios de prioridad.
        
    -   `-B 0`: Sin priority boost.
        
    -   `--jlist "0,50,0:10,50,0"`: Dos trabajos de 50 ms sin E/S.
        

**Análisis:**

-   Todos los trabajos permanecen en la misma cola y se alternan cada 10 ms (RR).
    
-   Sin cambios de prioridad ni boosts, el comportamiento es idéntico a Round Robin.

#### 4. Craft a workload with two jobs and scheduler parameters so that one job takes advantage of the older Rules 4a and 4b (turned on with the -S flag) to game the scheduler and obtain 99% of the CPU over a particular time interval.



Answer

-   **Explicación:**
    
    -   `-n 1`: Una sola cola.
        
    -   `-q 10`: Quantum de 10 ms.
        
    -   `-a 1000`: Allotment muy alto para evitar cambios de prioridad.
        
    -   `-B 0`: Sin priority boost.
        
    -   `--jlist "0,50,0:10,50,0"`: Dos trabajos de 50 ms sin E/S.
        

**Análisis:**

-   Todos los trabajos permanecen en la misma cola y se alternan cada 10 ms (RR).
    
-   Sin cambios de prioridad ni boosts, el comportamiento es idéntico a Round Robin.

#### 4. Craft a workload with two jobs and scheduler parameters so that one job takes advantage of the older Rules 4a and 4b (turned on with the -S flag) to game the scheduler and obtain 99% of the CPU over a particular time interval.



Answer

-   **Explicación:**
    
    -   `-n 1`: Una sola cola.
        
    -   `-q 10`: Quantum de 10 ms.
        
    -   `-a 1000`: Allotment muy alto para evitar cambios de prioridad.
        
    -   `-B 0`: Sin priority boost.
        
    -   `--jlist "0,50,0:10,50,0"`: Dos trabajos de 50 ms sin E/S.
        

**Análisis:**

-   Todos los trabajos permanecen en la misma cola y se alternan cada 10 ms (RR).
    
-   Sin cambios de prioridad ni boosts, el comportamiento es idéntico a Round Robin.

#### 4. Craft a workload with two jobs and scheduler parameters so that one job takes advantage of the older Rules 4a and 4b (turned on with the -S flag) to game the scheduler and obtain 99% of the CPU over a particular time interval.



Answer

-   **Explicación:**
    
    -   `-n 1`: Una sola cola.
        
    -   `-q 10`: Quantum de 10 ms.
        
    -   `-a 1000`: Allotment muy alto para evitar cambios de prioridad.
        
    -   `-B 0`: Sin priority boost.
        
    -   `--jlist "0,50,0:10,50,0"`: Dos trabajos de 50 ms sin E/S.
        

**Análisis:**

-   Todos los trabajos permanecen en la misma cola y se alternan cada 10 ms (RR).
    
-   Sin cambios de prioridad ni boosts, el comportamiento es idéntico a Round Robin.

#### 4. Craft a workload with two jobs and scheduler parameters so that one job takes advantage of the older Rules 4a and 4b (turned on with the -S flag) to game the scheduler and obtain 99% of the CPU over a particular time interval.



Answer

-   **Explicación:**
    
    -   `-n 1`: Una sola cola.
        
    -   `-q 10`: Quantum de 10 ms.
        
    -   `-a 1000`: Allotment muy alto para evitar cambios de prioridad.
        
    -   `-B 0`: Sin priority boost.
        
    -   `--jlist "0,50,0:10,50,0"`: Dos trabajos de 50 ms sin E/S.
        

**Análisis:**

-   Todos los trabajos permanecen en la misma cola y se alternan cada 10 ms (RR).
    
-   Sin cambios de prioridad ni boosts, el comportamiento es idéntico a Round Robin.

#### 4. Craft a workload with two jobs and scheduler parameters so that one job takes advantage of the older Rules 4a and 4b (turned on with the -S flag) to game the scheduler and obtain 99% of the CPU over a particular time interval.



Answer

-   **Explicación:**
    
    -   `-n 1`: Una sola cola.
        
    -   `-q 10`: Quantum de 10 ms.
        
    -   `-a 1000`: Allotment muy alto para evitar cambios de prioridad.
        
    -   `-B 0`: Sin priority boost.
        
    -   `--jlist "0,50,0:10,50,0"`: Dos trabajos de 50 ms sin E/S.
        

**Análisis:**

-   Todos los trabajos permanecen en la misma cola y se alternan cada 10 ms (RR).
    
-   Sin cambios de prioridad ni boosts, el comportamiento es idéntico a Round Robin.

#### 4. Craft a workload with two jobs and scheduler parameters so that one job takes advantage of the older Rules 4a and 4b (turned on with the -S flag) to game the scheduler and obtain 99% of the CPU over a particular time interval.



Answer

-   **Explicación:**
    
    -   `-n 1`: Una sola cola.
        
    -   `-q 10`: Quantum de 10 ms.
        
    -   `-a 1000`: Allotment muy alto para evitar cambios de prioridad.
        
    -   `-B 0`: Sin priority boost.
        
    -   `--jlist "0,50,0:10,50,0"`: Dos trabajos de 50 ms sin E/S.
        

**Análisis:**

-   Todos los trabajos permanecen en la misma cola y se alternan cada 10 ms (RR).
    
-   Sin cambios de prioridad ni boosts, el comportamiento es idéntico a Round Robin.

#### 4. Craft a workload with two jobs and scheduler parameters so that one job takes advantage of the older Rules 4a and 4b (turned on with the -S flag) to game the scheduler and obtain 99% of the CPU over a particular time interval.



Answer

-   **Explicación:**
    
    -   `-n 1`: Una sola cola.
        
    -   `-q 10`: Quantum de 10 ms.
        
    -   `-a 1000`: Allotment muy alto para evitar cambios de prioridad.
        
    -   `-B 0`: Sin priority boost.
        
    -   `--jlist "0,50,0:10,50,0"`: Dos trabajos de 50 ms sin E/S.
        

**Análisis:**

-   Todos los trabajos permanecen en la misma cola y se alternan cada 10 ms (RR).
    
-   Sin cambios de prioridad ni boosts, el comportamiento es idéntico a Round Robin.

#### 4. Craft a workload with two jobs and scheduler parameters so that one job takes advantage of the older Rules 4a and 4b (turned on with the -S flag) to game the scheduler and obtain 99% of the CPU over a particular time interval.



Answer

-   **Explicación:**
    
    -   `-n 1`: Una sola cola.
        
    -   `-q 10`: Quantum de 10 ms.
        
    -   `-a 1000`: Allotment muy alto para evitar cambios de prioridad.
        
    -   `-B 0`: Sin priority boost.
        
    -   `--jlist "0,50,0:10,50,0"`: Dos trabajos de 50 ms sin E/S.
        

**Análisis:**

-   Todos los trabajos permanecen en la misma cola y se alternan cada 10 ms (RR).
    
-   Sin cambios de prioridad ni boosts, el comportamiento es idéntico a Round Robin.

#### 4. Craft a workload with two jobs and scheduler parameters so that one job takes advantage of the older Rules 4a and 4b (turned on with the -S flag) to game the scheduler and obtain 99% of the CPU over a particular time interval.



Answer

-   **Explicación:**
    
    -   `-n 1`: Una sola cola.
        
    -   `-q 10`: Quantum de 10 ms.
        
    -   `-a 1000`: Allotment muy alto para evitar cambios de prioridad.
        
    -   `-B 0`: Sin priority boost.
        
    -   `--jlist "0,50,0:10,50,0"`: Dos trabajos de 50 ms sin E/S.
        

**Análisis:**

-   Todos los trabajos permanecen en la misma cola y se alternan cada 10 ms (RR).
    
-   Sin cambios de prioridad ni boosts, el comportamiento es idéntico a Round Robin.

#### 4. Craft a workload with two jobs and scheduler parameters so that one job takes advantage of the older Rules 4a and 4b (turned on with the -S flag) to game the scheduler and obtain 99% of the CPU over a particular time interval.



Answer

-   **Explicación:**
    
    -   `-n 1`: Una sola cola.
        
    -   `-q 10`: Quantum de 10 ms.
        
    -   `-a 1000`: Allotment muy alto para evitar cambios de prioridad.
        
    -   `-B 0`: Sin priority boost.
        
    -   `--jlist "0,50,0:10,50,0"`: Dos trabajos de 50 ms sin E/S.
        

**Análisis:**

-   Todos los trabajos permanecen en la misma cola y se alternan cada 10 ms (RR).
    
-   Sin cambios de prioridad ni boosts, el comportamiento es idéntico a Round Robin.

#### 4. Craft a workload with two jobs and scheduler parameters so that one job takes advantage of the older Rules 4a and 4b (turned on with the -S flag) to game the scheduler and obtain 99% of the CPU over a particular time interval.



Answer

-   **Explicación:**
    
    -   `-n 1`: Una sola cola.
        
    -   `-q 10`: Quantum de 10 ms.
        
    -   `-a 1000`: Allotment muy alto para evitar cambios de prioridad.
        
    -   `-B 0`: Sin priority boost.
        
    -   `--jlist "0,50,0:10,50,0"`: Dos trabajos de 50 ms sin E/S.
        

**Análisis:**

-   Todos los trabajos permanecen en la misma cola y se alternan cada 10 ms (RR).
    
-   Sin cambios de prioridad ni boosts, el comportamiento es idéntico a Round Robin.

#### 4. Craft a workload with two jobs and scheduler parameters so that one job takes advantage of the older Rules 4a and 4b (turned on with the -S flag) to game the scheduler and obtain 99% of the CPU over a particular time interval.



Answer

-   **Explicación:**
    
    -   `-n 1`: Una sola cola.
        
    -   `-q 10`: Quantum de 10 ms.
        
    -   `-a 1000`: Allotment muy alto para evitar cambios de prioridad.
        
    -   `-B 0`: Sin priority boost.
        
    -   `--jlist "0,50,0:10,50,0"`: Dos trabajos de 50 ms sin E/S.
        

**Análisis:**

-   Todos los trabajos permanecen en la misma cola y se alternan cada 10 ms (RR).
    
-   Sin cambios de prioridad ni boosts, el comportamiento es idéntico a Round Robin.

#### 4. Craft a workload with two jobs and scheduler parameters so that one job takes advantage of the older Rules 4a and 4b (turned on with the -S flag) to game the scheduler and obtain 99% of the CPU over a particular time interval.



Answer

-   **Explicación:**
    
    -   `-n 1`: Una sola cola.
        
    -   `-q 10`: Quantum de 10 ms.
        
    -   `-a 1000`: Allotment muy alto para evitar cambios de prioridad.
        
    -   `-B 0`: Sin priority boost.
        
    -   `--jlist "0,50,0:10,50,0"`: Dos trabajos de 50 ms sin E/S.
        

**Análisis:**

-   Todos los trabajos permanecen en la misma cola y se alternan cada 10 ms (RR).
    
-   Sin cambios de prioridad ni boosts, el comportamiento es idéntico a Round Robin.

#### 4. Craft a workload with two jobs and scheduler parameters so that one job takes advantage of the older Rules 4a and 4b (turned on with the -S flag) to game the scheduler and obtain 99% of the CPU over a particular time interval.



Answer

-   **Explicación:**
    
    -   `-n 1`: Una sola cola.
        
    -   `-q 10`: Quantum de 10 ms.
        
    -   `-a 1000`: Allotment muy alto para evitar cambios de prioridad.
        
    -   `-B 0`: Sin priority boost.
        
    -   `--jlist "0,50,0:10,50,0"`: Dos trabajos de 50 ms sin E/S.
        

**Análisis:**

-   Todos los trabajos permanecen en la misma cola y se alternan cada 10 ms (RR).
    
-   Sin cambios de prioridad ni boosts, el comportamiento es idéntico a Round Robin.

#### 4. Craft a workload with two jobs and scheduler parameters so that one job takes advantage of the older Rules 4a and 4b (turned on with the -S flag) to game the scheduler and obtain 99% of the CPU over a particular time interval.



Answer

-   **Explicación:**
    
    -   `-n 1`: Una sola cola.
        
    -   `-q 10`: Quantum de 10 ms.
        
    -   `-a 1000`: Allotment muy alto para evitar cambios de prioridad.
        
    -   `-B 0`: Sin priority boost.
        
    -   `--jlist "0,50,0:10,50,0"`: Dos trabajos de 50 ms sin E/S.
        

**Análisis:**

-   Todos los trabajos permanecen en la misma cola y se alternan cada 10 ms (RR).
    
-   Sin cambios de prioridad ni boosts, el comportamiento es idéntico a Round Robin.

#### 4. Craft a workload with two jobs and scheduler parameters so that one job takes advantage of the older Rules 4a and 4b (turned on with the -S flag) to game the scheduler and obtain 99% of the CPU over a particular time interval.



In [32]:
# Job A (víctima): Largo, sin E/S. Job B (gamer): Largo, llega un poco después (T=1),
# E/S muy frecuente, justo antes de acabar el quantum/allotment.
# 2 colas (-n 2), quantum 100ms (-q 100), allotment 1 slice (-a 1) -> Democión tras 100ms.
# Job B hace E/S cada 99ms. E/S corta (1ms, -i 1). Regla antigua (-S). Solución (-c).
!python mlfq.py --jlist 0,1000,0:1,1000,99 -n 2 -q 100 -a 1 -i 1 -S -c

Here is the list of inputs:
OPTIONS jobs 2
OPTIONS queues 2
OPTIONS allotments for queue  1 is   1
OPTIONS quantum length for queue  1 is 100
OPTIONS allotments for queue  0 is   1
OPTIONS quantum length for queue  0 is 100
OPTIONS boost 0
OPTIONS ioTime 1
OPTIONS stayAfterIO True
OPTIONS iobump False


For each job, three defining characteristics are given:
  startTime : at what time does the job enter the system
  runTime   : the total CPU time needed by the job to finish
  ioFreq    : every ioFreq time units, the job issues an I/O
              (the I/O takes ioTime units to complete)

Job List:
  Job  0: startTime   0 - runTime 1000 - ioFreq   0
  Job  1: startTime   1 - runTime 1000 - ioFreq  99


Execution Trace:

[ time 0 ] JOB BEGINS by JOB 0
[ time 0 ] Run JOB 0 at PRIORITY 1 [ TICKS 99 ALLOT 1 TIME 999 (of 1000) ]
[ time 1 ] JOB BEGINS by JOB 1
[ time 1 ] Run JOB 0 at PRIORITY 1 [ TICKS 98 ALLOT 1 TIME 998 (of 1000) ]
[ time 2 ] Run JOB 0 at PRIORITY 1 [ TICKS 97 ALLOT 1 TIME

Answer

**Análisis:**

-   Job A empieza y baja rápidamente a la cola Q0.

-   Job B llega (T=1) a la cola Q1 (la más alta).

-   Job B se ejecuta por 99ms. Justo antes de agotar su quantum/allotment de 100ms, realiza una E/S (que dura 1ms).

-   Debido a `-S`, cuando Job B termina su E/S, regresa a la cola Q1 (la más alta) con su allotment reseteado.

-   Job A solo se ejecuta durante el breve tiempo (1ms) que Job B está realizando la E/S.

-   Job B se ejecuta durante 99ms de cada ciclo de 100ms (99ms CPU + 1ms E/S). Por lo tanto, Job B obtiene el 99% de la CPU mientras está activo y compite con A. Calcula el uso de CPU en la traza para un intervalo largo (p.ej., de T=1 a T=1001) para confirmarlo.

#### 5. Given a system with a quantum length of 10 ms in its highest queue, how often would you have to boost jobs back to the highest priority level (with the `-B` flag) in order to guarantee that a single longrunning (and potentially-starving) job gets at least 5% of the CPU?



In [1]:
# N = 200 ms (calculado abajo).
# Job 0: Largo, propenso a inanición.
# Jobs 1-5: Cortos e interactivos, llegan T=1, acaparan CPU.
# Quantum 10ms (-q 10 o -Q 10,...), E/S 1ms (-i 1), Boost cada 200ms (-B 200).
!python mlfq.py --jlist 0,1000,0:1,30,1:1,30,1:1,30,1:1,30,1:1,30,1 -q 10 -i 1 -B 200 -c

Here is the list of inputs:
OPTIONS jobs 6
OPTIONS queues 3
OPTIONS allotments for queue  2 is   1
OPTIONS quantum length for queue  2 is  10
OPTIONS allotments for queue  1 is   1
OPTIONS quantum length for queue  1 is  10
OPTIONS allotments for queue  0 is   1
OPTIONS quantum length for queue  0 is  10
OPTIONS boost 200
OPTIONS ioTime 1
OPTIONS stayAfterIO False
OPTIONS iobump False


For each job, three defining characteristics are given:
  startTime : at what time does the job enter the system
  runTime   : the total CPU time needed by the job to finish
  ioFreq    : every ioFreq time units, the job issues an I/O
              (the I/O takes ioTime units to complete)

Job List:
  Job  0: startTime   0 - runTime 1000 - ioFreq   0
  Job  1: startTime   1 - runTime  30 - ioFreq   1
  Job  2: startTime   1 - runTime  30 - ioFreq   1
  Job  3: startTime   1 - runTime  30 - ioFreq   1
  Job  4: startTime   1 - runTime  30 - ioFreq   1
  Job  5: startTime   1 - runTime  30 - ioFreq   1




Answer

**Análisis y Cálculo:**

-   El objetivo es que el trabajo largo obtenga al menos el 5% (o 0.05) de la CPU.
-   El quantum en la cola más alta es de 10 ms.
-   Cuando ocurre un `boost` (cada `S` milisegundos, donde `S` es el valor de `-B`), el trabajo largo es movido a la cola más alta.
-   En el peor de los casos (muchos otros trabajos también son impulsados a la cola más alta), el trabajo largo obtendrá al menos _un_ quantum completo (10 ms) en la cola más alta inmediatamente después del boost, gracias al Round-Robin dentro de esa cola.
-   Para garantizar un 5% de CPU, este trabajo debe recibir sus 10 ms de ejecución dentro de un intervalo de tiempo `S` tal que `(tiempo de ejecución / intervalo total) >= 0.05`.
-   Es decir: `10 ms / S >= 0.05`.
-   Resolviendo para `S`: `S <= 10 ms / 0.05`.
-   `S <= 200 ms`.
-   Por lo tanto, necesitas realizar un `priority boost` (`-B`) **cada 200 milisegundos o menos** para garantizar que el trabajo largo obtenga al menos el 5% de la CPU. Ejecuta la simulación con `-B 200` y verifica en la traza que el Job 0 se ejecuta periódicamente. Podrías necesitar una simulación más larga (`-m` grande para el Job 0, o aumentar el tiempo total simulado si hubiera opción) para observar el porcentaje a largo plazo.





#### 6. One question that arises in scheduling is which end of a queue to add a job that just finished I/O; the -I flag changes this behavior
for this scheduling simulator. Play around with some workloads and see if you can see the effect of this flag.






In [34]:
# Con -I (trabajos van al frente de la cola tras E/S)
!python mlfq.py --jlist "0,50,5:10,50,5" -I -c

# Sin -I (trabajos van al final de la cola)
!python mlfq.py --jlist "0,50,5:10,50,5" -c

Here is the list of inputs:
OPTIONS jobs 2
OPTIONS queues 3
OPTIONS allotments for queue  2 is   1
OPTIONS quantum length for queue  2 is  10
OPTIONS allotments for queue  1 is   1
OPTIONS quantum length for queue  1 is  10
OPTIONS allotments for queue  0 is   1
OPTIONS quantum length for queue  0 is  10
OPTIONS boost 0
OPTIONS ioTime 5
OPTIONS stayAfterIO False
OPTIONS iobump True


For each job, three defining characteristics are given:
  startTime : at what time does the job enter the system
  runTime   : the total CPU time needed by the job to finish
  ioFreq    : every ioFreq time units, the job issues an I/O
              (the I/O takes ioTime units to complete)

Job List:
  Job  0: startTime   0 - runTime  50 - ioFreq   5
  Job  1: startTime  10 - runTime  50 - ioFreq   5


Execution Trace:

[ time 0 ] JOB BEGINS by JOB 0
[ time 0 ] Run JOB 0 at PRIORITY 2 [ TICKS 9 ALLOT 1 TIME 49 (of 50) ]
[ time 1 ] Run JOB 0 at PRIORITY 2 [ TICKS 8 ALLOT 1 TIME 48 (of 50) ]
[ time 2 ] Run JO

**Análisis:**

-   Con `-I`, los trabajos que completan E/S se ejecutan inmediatamente.
    
-   Sin `-I`, los trabajos esperan su turno en la cola actual.
    
-   Ejemplo: Si el trabajo A hace E/S, con `-I` retoma rápidamente; sin `-I`, espera detrás de otros en la misma cola.